In [1]:
import pandas as pd  
import numpy as np
from datetime import datetime as dt
import lppls
import collections

In [2]:
def fit_price(dataframe):
    time = [pd.Timestamp.toordinal(dt.strptime(str(t1), '%Y%m%d')) for t1 in dataframe.index]
    price = np.log(dataframe['S_DQ_ADJCLOSE'].values)
    observations = np.array([time, price])
    lppls_model = lppls.LPPLS(observations=observations)
    lppls_model.mp_compute_nested_fits(
                workers=6,
                window_size=80,
                smallest_window_size=5,
                outer_increment=2,
                inner_increment=5,
                max_searches=25,
    )
    return dataframe, lppls_model

In [3]:
def filt_indicator(dataframe, lppls_model, filter_conditions_config = None):
    res_df = lppls_model.compute_indicators(lppls_model.indicator_result, filter_conditions_config)
    res_df['time'] = [pd.Timestamp(pd.Timestamp.fromordinal(int(t1)).strftime('%Y-%m-%d %H:%M:%S')) for t1 in res_df['time']]
    dataframe['time'] = pd.to_datetime(dataframe['TRADE_DT'], format='%Y%m%d')
    dataframe=dataframe.merge(res_df, on='time')
    return dataframe

In [4]:
class ATRCalculator:
    def __init__(self, data, period):
        self.data = data
        self.period = period

    def calculate_tr(self):
        high = self.data['S_DQ_ADJHIGH']
        low = self.data['S_DQ_ADJLOW']
        close = self.data['S_DQ_ADJCLOSE']

        tr = pd.DataFrame()
        tr['HL'] = high - low
        tr['HC'] = abs(high - close.shift(1))
        tr['LC'] = abs(low - close.shift(1))

        tr['TR'] = tr[['HL', 'HC', 'LC']].max(axis=1)
        return tr

    def calculate_atr(self):
        tr = self.calculate_tr()
        atr = tr['TR'].rolling(self.period).mean().tail(1).item()
        return atr

In [16]:
def start(data):
    buy_price = data['S_DQ_ADJCLOSE'].tail(1).item()
    neg_conf = pos_conf = 0
    transac_portion = 1/2
    buy_times = 1
    pnl = 0
    position_dict[data['FUND_CODE'].unique().item()] += transac_portion
    return [neg_conf, pos_conf, transac_portion, buy_price, buy_times, pnl]
    
def calc_indicator(data):
        dataframe, lppls_model = fit_price(data)
        temp_df = filt_indicator(dataframe, lppls_model)
        return temp_df['neg_conf'].item(), temp_df['pos_conf'].item()

def next(data, res_df, period=20):
        ATR_val = ATRCalculator(data[-period:], period).calculate_atr()
        close_p = data.iloc[-1]['S_DQ_ADJCLOSE'].item()
        buy_times = res_df['buy_times'].item()
        buy_price = res_df['buy_price'].item()
        neg_conf, pos_conf = calc_indicator(data)
        transac_portion = 0

        if neg_conf > 0:
            if 0 <= buy_times < 5:
                transac_portion = 1/4
                buy_price = close_p
                buy_times += 1
            elif close_p < (buy_price - 2*ATR_val):
                transac_portion = -1
                buy_times = 0
        elif pos_conf > 0:
            transac_portion = -1/4
            buy_times = 0
        
        position = position_dict[res_df['FUND_CODE'].item()]
        last_p = data.iloc[-2]['S_DQ_ADJCLOSE'].item()
        pnl = position * (close_p / last_p - 1) * 100
        position_dict[res_df['FUND_CODE'].item()] += transac_portion
        
        return [neg_conf, pos_conf, transac_portion, buy_price, buy_times, pnl]
        

In [10]:
position_dict

{'159611.SZ': 0}

In [17]:
if __name__ == "__main__":
    # read in data
    etf_df = pd.read_csv('etf-trade-data-2020-2023.csv', encoding='gbk')
    etf_lst = etf_df['FUND_CODE'].unique()[:1]
    etf_df.index = etf_df['TRADE_DT']
    position_dict = dict.fromkeys(etf_lst, 0)

    # initialize output dataframe
    try:
        # if res_df already exist
        res_df = pd.read_csv('res_df.csv')
        for etf_code in etf_lst:
            single_etf = etf_df[etf_df['FUND_CODE'] == etf_code][-80:]
            if len(single_etf) >= 80:
                data_lst = [single_etf.tail(1)['TRADE_DT'].item(), etf_code]
                data_lst.extend(next(single_etf, res_df[res_df['FUND_CODE'] == etf_code].tail(1)))
                res_df = pd.concat([res_df, pd.DataFrame([data_lst], 
                    columns=['TRADE_DT', 'FUND_CODE', 'neg_conf', 'pos_conf', 'transac_portion', 'buy_price', 'buy_times', 'pnl (%)'])]).reset_index(drop=True)

    except FileNotFoundError:
        res_df = pd.DataFrame()

        for etf_code in etf_lst:
            # testing here to get last 100 lines of data for each etf to calculate history data for ATR stop signal
            single_etf = etf_df[etf_df['FUND_CODE'] == etf_code][-100:]
            t_range = len(single_etf) - 80
            if t_range > 0:
                # half position at the start
                data_lst = [single_etf.iloc[0:80].tail(1)['TRADE_DT'].item(), etf_code]
                data_lst.extend(start(single_etf))
                res_df = pd.concat([res_df, pd.DataFrame([data_lst], 
                    columns=['TRADE_DT', 'FUND_CODE', 'neg_conf', 'pos_conf', 'transac_portion', 'buy_price', 'buy_times', 'pnl (%)'])]).reset_index(drop=True)
                for i in range(1, t_range):
                    # iterate to get history data for ATR stop signal, if history data already given, 
                    # only need to call next() method once, no need to call start()
                    data_lst = [single_etf[i:i+80].tail(1)['TRADE_DT'].item(), etf_code]
                    data_lst.extend(next(single_etf[i:i+80], res_df[res_df['FUND_CODE'] == etf_code].tail(1)))
                    # for transac_portion column, value between 0 to 1 suggests the portion of available cash to buy, 
                    # the absolute value for a value between -1 to 0 suggests the portion of position to sell, 
                    # didn't count transaction fee here
                    res_df = pd.concat([res_df, pd.DataFrame([data_lst], 
                        columns=['TRADE_DT', 'FUND_CODE', 'neg_conf', 'pos_conf', 'transac_portion', 'buy_price', 'buy_times', 'pnl (%)'])]).reset_index(drop=True)
    

100%|██████████| 1/1 [00:11<00:00, 11.93s/it]
C:\Users\zhanq\AppData\Local\Temp\ipykernel_25040\1997069356.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['time'] = pd.to_datetime(dataframe['TRADE_DT'], format='%Y%m%d')
100%|██████████| 1/1 [00:11<00:00, 11.24s/it]
C:\Users\zhanq\AppData\Local\Temp\ipykernel_25040\1997069356.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['time'] = pd.to_datetime(dataframe['TRADE_DT'], format='%Y%m%d')
100%|██████████| 1/1 [00:10<00:00, 10.23s/it

In [18]:
res_df

,TRADE_DT,FUND_CODE,neg_conf,pos_conf,transac_portion,buy_price,buy_times,pnl
0,20230703,159611.SZ,0.000000,0.0,0.50,0.925,1,0.000000
1,20230704,159611.SZ,0.000000,0.0,0.00,0.925,1,-0.369198
2,20230705,159611.SZ,0.000000,0.0,0.00,0.925,1,-0.159405
3,20230706,159611.SZ,0.000000,0.0,0.00,0.925,1,-0.106610
4,20230707,159611.SZ,0.000000,0.0,0.00,0.925,1,0.427350
5,20230710,159611.SZ,0.000000,0.0,0.00,0.925,1,0.370763
6,20230711,159611.SZ,0.000000,0.0,0.00,0.925,1,-0.315457
7,20230712,159611.SZ,0.000000,0.0,0.00,0.925,1,-0.052910
8,20230713,159611.SZ,0.000000,0.0,0.00,0.925,1,0.211864
9,20230714,159611.SZ,0.000000,0.0,0.00,0.925,1,-0.580169


In [62]:
res_df.to_csv('res_df.csv', index=False)

In [63]:
pd.read_csv('res_df.csv')

,TRADE_DT,FUND_CODE,neg_conf,pos_conf,transac_portion,buy_price,buy_times
0,20230717,159611.SZ,0.0,0.0,0.5,0.925,1
1,20230718,159611.SZ,0.0,0.0,0.0,0.925,1
2,20230719,159611.SZ,0.0,0.0,0.0,0.925,1
3,20230720,159611.SZ,0.0,0.0,0.0,0.925,1
4,20230721,159611.SZ,0.0,0.0,0.0,0.925,1
5,20230724,159611.SZ,0.0,0.0,0.0,0.925,1
6,20230725,159611.SZ,0.0,0.0,0.0,0.925,1
7,20230726,159611.SZ,0.0,0.0,0.0,0.925,1
8,20230727,159611.SZ,0.0,0.0,0.0,0.925,1
9,20230728,159611.SZ,0.0,0.0,0.0,0.925,1
